In [1]:
# Import components, add data to pandas data frame
import pandas as pd
import numpy as np 
from pandasgui import show
from ipyvizzu import Data, Config, Style
from ipyvizzustory import Story, Slide, Step
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# Define common columns to load
columns_to_load = ["LocTypeID", "Sex", "Location", "AgeGrpStart", "Time", "ex"]

# Read and filter Male dataset (1950-2023)
df_male = pd.read_csv("WPP2024_Life_Table_Complete_Medium_Male_1950-2023.csv", usecols=columns_to_load, low_memory=False)
df_male_filtered = df_male[df_male["LocTypeID"] == 4].drop(columns=["LocTypeID"])

# Read and filter Female dataset (1950-2023)
df_female = pd.read_csv("WPP2024_Life_Table_Complete_Medium_Female_1950-2023.csv", usecols=columns_to_load, low_memory=False)
df_female_filtered = df_female[df_female["LocTypeID"] == 4].drop(columns=["LocTypeID"])

# Combine both datasets (1950-2023)
df_combined = pd.concat([df_male_filtered, df_female_filtered], ignore_index=True)

# Read and filter Male dataset (2024-2100) **only for 2024 and 2025**
df_male_future = pd.read_csv("WPP2024_Life_Table_Complete_Medium_Male_2024-2100.csv", usecols=columns_to_load, low_memory=False)
df_male_future_filtered = df_male_future[(df_male_future["LocTypeID"] == 4) & (df_male_future["Time"].isin([2024, 2025]))].drop(columns=["LocTypeID"])

# Read and filter Female dataset (2024-2100) **only for 2024 and 2025**
df_female_future = pd.read_csv("WPP2024_Life_Table_Complete_Medium_Female_2024-2100.csv", usecols=columns_to_load, low_memory=False)
df_female_future_filtered = df_female_future[(df_female_future["LocTypeID"] == 4) & (df_female_future["Time"].isin([2024, 2025]))].drop(columns=["LocTypeID"])

# Append 2024-2025 data to df_combined
df_combined = pd.concat([df_combined, df_male_future_filtered, df_female_future_filtered], ignore_index=True)


In [3]:
#show(df_combined)

In [4]:
print(df_combined.dtypes)


Location        object
Time             int64
Sex             object
AgeGrpStart      int64
ex             float64
dtype: object


In [5]:
# Copy the combined dataset
df_wide = df_combined.copy()

# Calculate BirthYear BEFORE melting
df_wide["BirthYear"] = df_wide["Time"] - df_wide["AgeGrpStart"]

# Calculate total (Age + Remaining Years)
df_wide["Total"] = df_wide["AgeGrpStart"] + df_wide["ex"]

# Calculate percentages BEFORE melting
df_wide["Age_Percentage"] = (df_wide["AgeGrpStart"] / df_wide["Total"]) * 100
df_wide["Remaining_Years_Percentage"] = (df_wide["ex"] / df_wide["Total"]) * 100

# **Ensure the percentages add up to 100%**
df_wide["Age_Percentage"] = df_wide["Age_Percentage"].round(2)
df_wide["Remaining_Years_Percentage"] = (100 - df_wide["Age_Percentage"]).round(2)

# Melt the DataFrame to long format (excluding percentage values)
df_vizzu = df_wide.melt(
    id_vars=["Sex", "Location", "Time", "BirthYear"],  # Keep BirthYear
    value_vars=["AgeGrpStart", "ex"],  # Melt Age and Remaining Years
    var_name="Category",
    value_name="Value"
)

# Replace category names
df_vizzu["Category"] = df_vizzu["Category"].replace({
    "AgeGrpStart": "Age", 
    "ex": "Remaining Years"
})

# Create a lookup table for percentages BEFORE melting
percentage_lookup = df_wide.melt(
    id_vars=["Sex", "Location", "Time", "BirthYear"],  # Keep BirthYear
    value_vars=["Age_Percentage", "Remaining_Years_Percentage"],
    var_name="Category",
    value_name="Percentage"
)

# Replace percentage category names
percentage_lookup["Category"] = percentage_lookup["Category"].replace({
    "Age_Percentage": "Age",
    "Remaining_Years_Percentage": "Remaining Years"
})

# Merge the precomputed percentages into `df_vizzu`
df_vizzu = df_vizzu.merge(percentage_lookup, on=["Sex", "Location", "Time", "BirthYear", "Category"], how="left")

# Format Percentage column correctly
df_vizzu["Percentage"] = df_vizzu["Percentage"].apply(lambda x: f"{x:.2f}%" if pd.notna(x) else None)


In [6]:
show(df_vizzu)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI


In [7]:
df_vizzu.to_csv("life_expectancy.csv", index=False)

In [8]:
# Define filtering parameters
target_country = "Hungary"
target_birth_year = 1981

# Apply filtering conditions
df_filtered = df_vizzu[
    ((df_vizzu["BirthYear"] == target_birth_year) & 
     (df_vizzu["Location"] == target_country)) 
    |
    ((df_vizzu["BirthYear"] == target_birth_year) & 
     (df_vizzu["Category"] == "Remaining Years") & 
     (df_vizzu["Time"] == target_birth_year))
]

df_filtered = df_filtered.sort_values(by=["Sex", "Location", "Time"]).reset_index(drop=True)

# Save the filtered dataset
df_filtered.to_csv("filtered_data.csv", index=False)


In [9]:
show(df_filtered)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
